In [3]:
import project_path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import yaml
import json
import pathlib

In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
from eye_tracking.post_processing import get_weights, get_followup
CONFIG_PATH = "../config/eye_v11.yaml"

In [4]:
m = np.ones((7, 3, 3))
m

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]])

In [19]:
scale_weights = np.arange(1, m.shape[0] + 1)
scale_weights

array([1, 2, 3, 4, 5, 6, 7])

In [20]:
# expand two dimension so to have size 7x3x3
scale_weights = np.repeat(scale_weights[:, np.newaxis], 3, axis=1)
scale_weights = np.repeat(scale_weights[:, :, np.newaxis], 3, axis=2)
scale_weights

array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]],

       [[3, 3, 3],
        [3, 3, 3],
        [3, 3, 3]],

       [[4, 4, 4],
        [4, 4, 4],
        [4, 4, 4]],

       [[5, 5, 5],
        [5, 5, 5],
        [5, 5, 5]],

       [[6, 6, 6],
        [6, 6, 6],
        [6, 6, 6]],

       [[7, 7, 7],
        [7, 7, 7],
        [7, 7, 7]]])

In [21]:
scale_weights.shape

(7, 3, 3)

In [22]:
m * scale_weights

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]],

       [[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]],

       [[5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.]],

       [[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.]],

       [[7., 7., 7.],
        [7., 7., 7.],
        [7., 7., 7.]]])

In [1]:
# Check if CodeGen and GPTJ have the same tokenizer

In [13]:
from attwizard.script.utils import load_json_file
task_name = 'Nqueens_Q3.cs'
METADATA_CODEGEN = f'../data/model_output/exp_v10/codegen/metadata/{task_name}_0_Salesforce_codegen-16B-multi.json'
METADATA_GPTJ = f'../data/model_output/exp_v10/gpt-j/metadata/{task_name}_0_EleutherAI_gpt-j-6B.json'
metadata_codegen = load_json_file(METADATA_CODEGEN)
metadata_gptj = load_json_file(METADATA_GPTJ)

In [14]:
metadata_gptj.keys()

dict_keys(['tokens_all', 'tokens_prompt', 'tokens_generated', 'text_generated', 'text_prompt', 'generation_time_seconds', 'source_file', 'mode_name', 'config_options'])

In [15]:
gptj_tokens = metadata_gptj['tokens_prompt']
codegen_tokens = metadata_codegen['tokens_prompt']
print("GPT-J tokens: ", gptj_tokens)
print("Codegen tokens: ", codegen_tokens)

GPT-J tokens:  ['/', '********************************', '********************************', '********************************', '*/', 'ĊĊ', 'Ċ', '/*', 'Ċ', '#', 'ĠThe', 'Ġn', '-', 'que', 'ens', 'Ġpuzzle', 'Ġis', 'Ġthe', 'Ġproblem', 'Ġof', 'Ġplacing', 'Ġn', 'Ġqueens', 'Ġon', 'Ġan', 'Ġn', 'Ġx', 'Ġn', 'Ġchess', 'board', 'Ċ', 'Ċ', '#', 'Ġsuch', 'Ġthat', 'Ġno', 'Ġtwo', 'Ġqueens', 'Ġattack', 'Ġeach', 'Ġother', '.', 'Ċ', 'Ċ', '#', 'ĠThe', 'Ġalgorithm', 'Ġbelow', 'Ġsolves', 'Ġthe', 'Ġfollowing', 'Ġproblem', '.', 'Ċ', '*/', 'ĊĊ', 'Ċ', 'using', 'ĠSystem', ';', 'Ċ', 'Ċ', 'class', 'ĠG', 'FG', 'Ċ', '{', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġread', 'only', 'Ġint', 'ĠN', 'Ġ=', 'Ġ4', ';', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġvoid', 'Ġprint', 'Solution', '(', 'int', 'Ġ[', ',', ']', 'board', ')', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ{', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġfor', 'Ġ(', 'int', 'Ġi', 'Ġ=', 'Ġ0', ';', 'Ġi', 'Ġ<', 'ĠN', ';', 'Ġi', '++)', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ{', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ

In [17]:
for i in range(min(len(gptj_tokens), len(codegen_tokens))):
    if gptj_tokens[i] != codegen_tokens[i]:
        print(i, gptj_tokens[i], codegen_tokens[i])
    else: 
        print(i, gptj_tokens[i], codegen_tokens[i], "OK")

0 / / OK
1 ******************************** ******************************** OK
2 ******************************** ******************************** OK
3 ******************************** ******************************** OK
4 */ */ OK
5 ĊĊ ĊĊ OK
6 Ċ Ċ OK
7 /* /* OK
8 Ċ Ċ OK
9 # # OK
10 ĠThe ĠThe OK
11 Ġn Ġn OK
12 - - OK
13 que que OK
14 ens ens OK
15 Ġpuzzle Ġpuzzle OK
16 Ġis Ġis OK
17 Ġthe Ġthe OK
18 Ġproblem Ġproblem OK
19 Ġof Ġof OK
20 Ġplacing Ġplacing OK
21 Ġn Ġn OK
22 Ġqueens Ġqueens OK
23 Ġon Ġon OK
24 Ġan Ġan OK
25 Ġn Ġn OK
26 Ġx Ġx OK
27 Ġn Ġn OK
28 Ġchess Ġchess OK
29 board board OK
30 Ċ Ċ OK
31 Ċ Ċ OK
32 # # OK
33 Ġsuch Ġsuch OK
34 Ġthat Ġthat OK
35 Ġno Ġno OK
36 Ġtwo Ġtwo OK
37 Ġqueens Ġqueens OK
38 Ġattack Ġattack OK
39 Ġeach Ġeach OK
40 Ġother Ġother OK
41 . . OK
42 Ċ Ċ OK
43 Ċ Ċ OK
44 # # OK
45 ĠThe ĠThe OK
46 Ġalgorithm Ġalgorithm OK
47 Ġbelow Ġbelow OK
48 Ġsolves Ġsolves OK
49 Ġthe Ġthe OK
50 Ġfollowing Ġfollowing OK
51 Ġproblem Ġproblem OK
52 . . OK
53 Ċ Ċ OK
54 */ */ O

In [18]:
from transformers import AutoTokenizer, GPTNeoForCausalLM, GPTJForCausalLM, AutoModelForCausalLM
from attwizard.script.utils import get_model_folder_path

model_folder_path = get_model_folder_path(
    model_folder='/mnt/huggingface_models/',
    hugging_short_repo='Salesforce/codegen-16B-multi'
)
tokenizer_codegen = AutoTokenizer.from_pretrained(model_folder_path)


In [19]:

model_folder_path = get_model_folder_path(
    model_folder='/mnt/huggingface_models/',
    hugging_short_repo='EleutherAI/gpt-j-6B'
)
tokenizer_gptj = AutoTokenizer.from_pretrained(model_folder_path)


In [32]:
raw_text = metadata_gptj['text_prompt']

In [42]:
# parse the raw text with the two tokenizers
tmp = tokenizer_gptj(raw_text, return_tensors="pt")
tokens = tmp['input_ids'][0].tolist()
tokens_gptj = tokenizer_gptj.convert_ids_to_tokens(tokens)
tokens_gptj_decoded = list([tokenizer_gptj.decode([t]) for t in tokens])



In [41]:
raw_text = """# ***************************************************************************\n\n# The following code deals with the creation of a tree data structure.\n\n\nclass Node:\n\n    def __init__(self, data):\n\n        self.data = data\n        self.left = None\n        self.right = None\n\n\ndef constructTreeUtil(pre: list, post: list,\n                      l: int, h: int,\n                      size: int) -> Node:\n    global preIndex\n\n    if (preIndex >= size or l > h):\n        return None\n\n    root = Node(pre[preIndex])\n    preIndex += 1\n\n    if (l == h or preIndex >= size):\n        return root\n\n    i = l\n    while i <= h:\n        if (pre[preIndex] == post[i]):\n            break\n\n        i += 1\n\n    if (i <= h):\n        root.left = constructTreeUtil(pre, post,\n                                      l, i, size)\n        root.right = constructTreeUtil(pre, post,\n                                       i + 1, h-1,\n                                       size)\n\n    return root\n\n\ndef constructTree(pre: list,\n                  post: list,\n                  size: int) -> Node:\n\n    global preIndex\n\n    return constructTreeUtil(pre, post, 0,\n                             size - 1, size)\n\n\ndef printInorder(node: Node) -> None:\n\n    if (node is None):\n        return\n\n    printInorder(node.left)\n    print(node.data, end=\" \")\n\n    printInorder(node.right)\n\n\nif __name__ == \"__main__\":\n\n    pre = [1, 2, 4, 8, 9, 5, 3, 6, 7]\n    post = [8, 9, 4, 5, 2, 6, 7, 3, 1]\n    size = len(pre)\n\n    preIndex = 0\n\n    root = constructTree(pre, post, size)\n\n    print(\"Inorder traversal of \"\n          \"the constructed tree: \")\n\n    printInorder(root)\n\n\n# Question: Under which conditions could the check `if i <= h` in\n\n# `constructTreeUtil` be false?\n\n# Answer:"""

In [37]:

tmp = tokenizer_codegen(raw_text, return_tensors="pt")
tokens = tmp['input_ids'][0].tolist()
tokens_codegen = tokenizer_codegen.convert_ids_to_tokens(tokens)
# decode tokens
tokens_codegen_decoded = list([tokenizer_codegen.decode([t]) for t in tokens])

In [38]:
tokens_codegen_decoded[:10]

['#',
 ' ********************************',
 '********************************',
 '********',
 '***',
 '\n',
 '\n',
 '#',
 ' The',
 ' following']

In [24]:

for i in range(min(len(tokens_gptj), len(tokens_codegen))):
    if tokens_gptj[i] != tokens_codegen[i]:
        print(i, tokens_gptj[i], tokens_codegen[i])
    else: 
        print(i, tokens_gptj[i], tokens_codegen[i], "OK")

0 / / OK
1 ******************************** ******************************** OK
2 ******************************** ******************************** OK
3 ******************************** ******************************** OK
4 */ */ OK
5 ĊĊ ĊĊ OK
6 Ċ Ċ OK
7 /* /* OK
8 Ċ Ċ OK
9 # # OK
10 ĠThe ĠThe OK
11 Ġn Ġn OK
12 - - OK
13 que que OK
14 ens ens OK
15 Ġpuzzle Ġpuzzle OK
16 Ġis Ġis OK
17 Ġthe Ġthe OK
18 Ġproblem Ġproblem OK
19 Ġof Ġof OK
20 Ġplacing Ġplacing OK
21 Ġn Ġn OK
22 Ġqueens Ġqueens OK
23 Ġon Ġon OK
24 Ġan Ġan OK
25 Ġn Ġn OK
26 Ġx Ġx OK
27 Ġn Ġn OK
28 Ġchess Ġchess OK
29 board board OK
30 Ċ Ċ OK
31 Ċ Ċ OK
32 # # OK
33 Ġsuch Ġsuch OK
34 Ġthat Ġthat OK
35 Ġno Ġno OK
36 Ġtwo Ġtwo OK
37 Ġqueens Ġqueens OK
38 Ġattack Ġattack OK
39 Ġeach Ġeach OK
40 Ġother Ġother OK
41 . . OK
42 Ċ Ċ OK
43 Ċ Ċ OK
44 # # OK
45 ĠThe ĠThe OK
46 Ġalgorithm Ġalgorithm OK
47 Ġbelow Ġbelow OK
48 Ġsolves Ġsolves OK
49 Ġthe Ġthe OK
50 Ġfollowing Ġfollowing OK
51 Ġproblem Ġproblem OK
52 . . OK
53 Ċ Ċ OK
54 */ */ O

In [26]:
# Compare followup with followup scaled  

In [28]:
PATH_FOLLOWUP = '../data/model_output/exp_v10/codegen/followup/hannoi_Q2.cpp_0_Salesforce_codegen-16B-multi.npy'
PATH_FOLLOWUP_SCALED = '../data/model_output/exp_v10/codegen/followup_scaled/hannoi_Q2.cpp_0_Salesforce_codegen-16B-multi.npy'
matrix_followup = np.load(PATH_FOLLOWUP)
matrix_followup_scaled = np.load(PATH_FOLLOWUP_SCALED)


In [29]:

# compare the two matrices 
N_ELEMENTS = 5
print("Followup matrix: ", matrix_followup[:N_ELEMENTS, :N_ELEMENTS])
print("Followup scaled matrix: ", matrix_followup_scaled[:N_ELEMENTS, :N_ELEMENTS])


Followup matrix:  [[32.4628   16.344732 14.723575 14.251423 16.741604]
 [16.028666 29.85003  28.343182 25.98302  25.506956]
 [14.229357 28.137642 29.533432 27.16568  26.7729  ]
 [13.592943 25.874996 27.517221 29.658264 28.151424]
 [15.990891 25.38924  26.580202 28.204033 30.125004]]
Followup scaled matrix:  [[557.23287785 251.11613357 227.39294292 213.59032701 288.2018964 ]
 [252.55994552 508.8562358  485.72750235 428.65722036 411.83441818]
 [229.01778363 482.46649998 508.53773558 460.25386178 444.14677864]
 [213.17125931 432.5248903  469.8751654  504.7467038  468.48590124]
 [279.22762531 410.41920269 438.04380381 467.94382888 508.20863843]]
